In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from transformers import BertTokenizer, BertModel
import pickle

import spacy
import pandas as pd

c:\Users\Jordi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Carreguem dades
DF = pd.read_pickle("data/datos_preprocesados.pkl")

Creem un dataframe amb el nom i la descripció

In [ ]:
df = DF[["Name","Description"]]
df = df.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)
DF = DF.drop(columns=["Description"])
#print(df)

                                      Name  \
0                            TRAVELPERK SL   
1                     EVERYDAY SOFTWARE SL   
2               SATELIO IOT SERVICES, S.L.   
3    DEEPULL DIAGNOSTICS SOCIEDAD LIMITADA   
4                       BELVO TECHNOLOGIES   
..                                     ...   
346                      GENTECH 360, S.L.   
347                         ZOUNDREAM S.L.   
348                     SILT DIGITAL ID SL   
349            CONSTRUMARKET DIGITAL, S.L.   
350                        SYRA COFFE S.L.   

                                           Description  
0    Travelperk is a business travel platform pione...  
1    We've developed a free software that let hr de...  
2                       Iot connectivity via satellite  
3    Deepull is a development-stage clinical diagno...  
4    An open banking platform that enables any comp...  
..                                                 ...  
346  GAMIUM is a technological company that has 

TF-idf

In [4]:
# Crear el vectoritzador
vectorizer = TfidfVectorizer()

# Aplicar TF-IDF sobre la columna 'Description'
tfidf_matrix = vectorizer.fit_transform(df['Description'])

# Convertir la matriu en un DataFrame per veure-ho clar
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Afegir l'ID o index per context
tfidf_df['Name'] = df['Name']

In [ ]:
#tfidf_df

,000,01,011h,100,101,12,120,15,17,1st,...,zero,zerod,zones,zoundream,zymvol,àmbit,és,ús,útils,Name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,TRAVELPERK SL
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,EVERYDAY SOFTWARE SL
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,"SATELIO IOT SERVICES, S.L."
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,DEEPULL DIAGNOSTICS SOCIEDAD LIMITADA
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,BELVO TECHNOLOGIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,"GENTECH 360, S.L."
347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.14802,0.0,0.0,0.0,0.0,0.0,ZOUNDREAM S.L.
348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,SILT DIGITAL ID SL
349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,"CONSTRUMARKET DIGITAL, S.L."


In [6]:
DF_tfidf = DF.merge(tfidf_df, on="Name", how="left")

Word2Vec

In [7]:
# Carrega el model amb embeddings
nlp = spacy.load("en_core_web_md")

# Exemple: df = pd.DataFrame({'Name': [...], 'Description': [...]})

# Funció per calcular vector mitjà de la frase
def sentence_vector(text):
    doc = nlp(text)
    return doc.vector

# Aplicar a cada descripció
sentence_vecs = df['Description'].astype(str).apply(sentence_vector)
sentence_vecs_df = pd.DataFrame(sentence_vecs.tolist())
sentence_vecs_df.insert(0, 'Name', df['Name'])

In [ ]:
#sentence_vecs_df

,Name,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,TRAVELPERK SL,-0.671870,0.275241,-0.049008,-0.086101,-0.043683,-0.025181,0.080174,-0.073714,-0.006702,...,-0.280291,0.126264,-0.047731,0.020952,-0.034946,0.031820,0.005075,-0.107811,0.073892,0.019900
1,EVERYDAY SOFTWARE SL,-0.732906,0.239169,-0.155859,-0.075722,-0.151827,-0.012113,0.097738,-0.123799,0.033677,...,-0.280113,0.087424,0.012737,0.024525,0.050097,0.049679,-0.043553,-0.113525,0.015029,-0.001074
2,"SATELIO IOT SERVICES, S.L.",-0.739608,0.287878,-0.053394,-0.053947,-0.267882,0.202295,0.002713,0.155575,0.166944,...,-0.256861,0.080157,0.295372,0.244678,0.093392,-0.013722,0.139957,-0.167108,0.093840,0.091205
3,DEEPULL DIAGNOSTICS SOCIEDAD LIMITADA,-0.642529,0.353359,-0.120071,-0.030493,-0.150295,-0.193156,0.098549,-0.069624,0.269522,...,-0.110281,0.027545,0.067637,-0.013345,0.010986,0.069888,-0.113573,-0.006820,0.109550,0.098258
4,BELVO TECHNOLOGIES,-0.684588,0.218378,-0.145700,0.013130,-0.109961,0.024288,0.024703,-0.076624,-0.055328,...,-0.375975,0.245096,-0.049540,-0.003580,0.011015,-0.080150,-0.023859,-0.147787,0.019146,0.051218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,"GENTECH 360, S.L.",-0.673058,0.127046,-0.153269,-0.123463,0.049642,0.003655,0.104449,-0.005098,0.073162,...,-0.281607,0.128927,-0.012306,-0.013336,0.053761,0.197000,-0.024922,-0.139407,0.013979,0.020930
347,ZOUNDREAM S.L.,-0.709188,0.155533,-0.214801,-0.074530,-0.167928,-0.070445,0.093160,-0.124887,-0.006357,...,-0.189731,0.098998,-0.084314,-0.002953,0.054943,0.135561,-0.009826,-0.118422,0.016086,-0.007513
348,SILT DIGITAL ID SL,-0.719298,0.054801,-0.057166,-0.160304,-0.050054,0.228963,0.026765,-0.273036,0.139362,...,-0.283296,-0.023505,-0.003614,0.085212,0.129370,-0.147794,0.006421,0.080255,-0.102202,0.334644
349,"CONSTRUMARKET DIGITAL, S.L.",-0.716900,0.348896,-0.044338,-0.153881,0.033069,0.050728,-0.054802,-0.071740,-0.053992,...,-0.449350,0.111507,0.035345,-0.008242,-0.194957,-0.024241,-0.036598,-0.140943,0.106802,0.044366


In [9]:
DF_W2V = DF.merge(sentence_vecs_df, on="Name", how="left")

Transformers Bert

In [10]:
# Suposem que tens el DataFrame df amb 'Description' i 'Name'
descriptions = df['Description'].astype(str).tolist()

# Carregar model i tokenizer BERT (base uncased, pots canviar a model en català si vols)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Funció per obtenir l'embedding d'una frase (mitjana dels tokens)
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # outputs.last_hidden_state té la representació per cada token
    # Fem la mitjana sobre els tokens per tenir un vector fix (1 x 768)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Generar embeddings per tots els textos (pot ser lent si és gran)
embeddings = [get_embedding(text) for text in descriptions]

# Convertir a DataFrame, 768 columnes per BERT base
embeddings_df = pd.DataFrame(embeddings)

# Afegir la columna Name
embeddings_df.insert(0, 'Name', df['Name'].values)

In [11]:
#embeddings_df

In [12]:
DF_TBert = DF.merge(embeddings_df, on="Name", how="left")


In [ ]:
DF_tfidf

In [ ]:
DF_W2V

In [ ]:
DF_TBert

In [ ]:
DF_tfidf.to_pickle(path="data/dades_TF-idf.pkl")
DF_W2V.to_pickle(path="data/dades_W2V.pkl")
DF_TBert.to_pickle(path="data/dades_TBert.pkl")

ACM

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import prince

# Suposem que tens el DataFrame df amb 'Description' i 'Name'

# 1. Vectoritzar les frases en forma binària (presència/absència)
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['Description'].astype(str))

# Convertir a DataFrame per comoditat
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# 2. Aplicar ACM amb prince
mca = prince.MCA(n_components=10, n_iter=3, copy=True, check_input=True, engine='sklearn', random_state=42)
mca = mca.fit(X_df)

# 3. Coordenades de les paraules (columnes)
word_coords = mca.column_coordinates(X_df)

# 4. Funció per obtenir el vector de frase com mitjana vectors paraules
def sentence_vector(sentence):
    words = sentence.lower().split()
    valid_words = [w for w in words if w in word_coords.index]
    if not valid_words:
        return [0]*word_coords.shape[1]
    return word_coords.loc[valid_words].mean(axis=0).values

# 5. Aplicar a totes les frases
sentence_vectors = df['Description'].astype(str).apply(sentence_vector).tolist()

# 6. Crear DataFrame amb resultats
sentence_vectors_df = pd.DataFrame(sentence_vectors)
sentence_vectors_df.insert(0, 'Name', df['Name'].values)

In [ ]:
sentence_vectors_df